In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
import pandas as pd
from time import sleep

In [2]:
url = 'https://www.thegioididong.com/dtdd'

driver = webdriver.Chrome()
driver.get(url)

In [3]:
tablet_elm = driver.find_element(By.XPATH, '/html/body/header/div[2]/div/ul/li[6]')
tablet_elm

<selenium.webdriver.remote.webelement.WebElement (session="53623901c9fb65c6ef43947cbd919639", element="f.79CA07ED548E52B3DADBBC6305ECDC5A.d.D23B014F190FB705868ADA7E98448209.e.11")>

In [4]:
tablet_elm.click()

In [5]:
show_more_elm = driver.find_element(By.XPATH, '//*[@id="categoryPage"]/div[1]/div[2]/a/strong')
show_more_elm

<selenium.webdriver.remote.webelement.WebElement (session="53623901c9fb65c6ef43947cbd919639", element="f.79CA07ED548E52B3DADBBC6305ECDC5A.d.852881464E2E1538184DBE6C041BAC6D.e.159")>

In [6]:
while True:
    try:
        show_more_elm.click()
        sleep(2)
    except:
        print('We are at the end of the page!')
        break

We are at the end of the page!


### Lấy danh sách sản phẩm

In [7]:
listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'ajaxed')
len(product_list)

52

#### Tạo Func lấy thông 1 tin sản phẩm

In [8]:
def get_product_info(product_elm):
    phone_name = product_elm.find_element(By.XPATH, './/h3').text

    try:
        phone_price = product_elm.find_element(By.XPATH, './/strong[@class="price"]').text
    except:
        phone_price = "N/A"

    result = {
        'name': phone_name,
        'price': phone_price
    }
    return result

In [9]:
# Hàm lấy review của tất cả điện thoại
all_reviews = []
tmp = 0
def getReviewAllPhone(product_list):
    for item in product_list:
        product_info = get_product_info(item)
        item.click()
        sleep(2)

        try:
            # Nếu mà có readmore thì mình bấm luôn
            read_more_elm = driver.find_element(By.XPATH, '/html/body/section/div[2]/div[1]/div[10]/div[2]/div/div/div[6]/div/a')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        all_reviews.append({
                            'product_name': product_info['name'],
                            'product_price': product_info['price'],
                            'review': review.text
                        })

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None # Init var
                        for page in list_page:
                            if page.text.strip() == "»":
                                next_page = page
                                break

                        if next_page: # if next_page exist
                            next_page.click() # forward
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            # Lấy xong hết review của trang cuối cùng của sản phẩm
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                all_reviews.append({
                                    'product_name': product_info['name'],
                                    'product_price': product_info['price'],
                                    'review': review.text
                                })
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:

            # Nếu có review nhưng chỉ có ===>> 1 TRANG
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            # Lấy xong thì back về
            driver.back()
            sleep(2)
            listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
            product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'ajaxed')

In [10]:
getReviewAllPhone(product_list)

Không còn trang tiếp theo!
Không còn trang tiếp theo!
Không còn trang tiếp theo!


In [11]:
all_reviews

[{'product_name': 'OPPO Pad 3 8GB/256GB',
  'product_price': '11.990.000₫',
  'review': 'Tỷ lệ màn hình đẹp.. chất liệu hiển thị tốt'},
 {'product_name': 'iPad 10 WiFi 64GB',
  'product_price': '9.490.000₫',
  'review': 'Đang chs game máy bị giảm độ sáng đột ngột bật lại k đc là bị gì . Đã tắt true tone vs độ sáng tự động'},
 {'product_name': 'iPad 10 WiFi 64GB',
  'product_price': '9.490.000₫',
  'review': 'Mới mua hôm 15/2 . Nay máy lỗi sạc ko vào, phải gửi bảo hành hãng, thật khó chịu. Từ khi đặt hàng tới khi nhận hàng và trải nghiệm sản phẩm không có chút vui vẻ nào.'},
 {'product_name': 'iPad 10 WiFi 64GB',
  'product_price': '9.490.000₫',
  'review': 'Bên mình có thu iPad 10 64GB wifi để đổi qua iPad Air 6 128G không ạ Giá như thế nào'},
 {'product_name': 'iPad 10 WiFi 64GB',
  'product_price': '9.490.000₫',
  'review': 'Máy 23/9 thu lại bao nhiêu'},
 {'product_name': 'iPad 10 WiFi 64GB',
  'product_price': '9.490.000₫',
  'review': 'Màn hình đúng là không ổn. Tiền nào của nấy th

In [12]:
df = pd.DataFrame(all_reviews)
# Lưu file
df.to_csv("E:\\study\\tu_hoc\\DE\\CRAWL_DATA\\review_data\\data\\cmt_data.xlsx", index=False)